# 01-clean.ipynb  
## Limpeza e integração dos dados  
**Objetivo**: tratar duplicados, valores nulos, criar métricas derivadas e unir todos os datasets em um DataFrame único para análises posteriores.


In [21]:
# 1. Imports gerais
import pandas as pd
import numpy as np

In [22]:
import pandas as pd

reviews = pd.read_csv(
    "../data/olist_order_reviews_dataset.csv",
    usecols=[
        "order_id",
        "review_score",
        "review_creation_date",
        "review_answer_timestamp"
    ],
    low_memory=False
)

print("reviews (antes do parse):", reviews.shape)
print("Nulos por coluna:\n", reviews.isna().sum(), "\n")
display(reviews.head())

# 2) Converte as colunas de data em datetime (vetorizado e mais rápido que parse_dates na leitura)
reviews["review_creation_date"]      = pd.to_datetime(reviews["review_creation_date"],      infer_datetime_format=True)
reviews["review_answer_timestamp"]  = pd.to_datetime(reviews["review_answer_timestamp"],  infer_datetime_format=True)

print("\nreviews (após parse):")
print("Tipos:", reviews.dtypes[["review_creation_date", "review_answer_timestamp"]])
display(reviews.head())


reviews (antes do parse): (99224, 4)
Nulos por coluna:
 order_id                   0
review_score               0
review_creation_date       0
review_answer_timestamp    0
dtype: int64 



,order_id,review_score,review_creation_date,review_answer_timestamp
0,73fc7af87114b39712e6da79b0a377eb,4,2018-01-18 00:00:00,2018-01-18 21:46:59
1,a548910a1c6147796b98fdf73dbeba33,5,2018-03-10 00:00:00,2018-03-11 03:05:13
2,f9e4b658b201a9f2ecdecbb34bed034b,5,2018-02-17 00:00:00,2018-02-18 14:36:24
3,658677c97b385a9be170737859d3511b,5,2017-04-21 00:00:00,2017-04-21 22:02:06
4,8e6bfb81e283fa7e4f11123a3fb894f1,5,2018-03-01 00:00:00,2018-03-02 10:26:53



reviews (após parse):
Tipos: review_creation_date       datetime64[ns]
review_answer_timestamp    datetime64[ns]
dtype: object


/tmp/ipykernel_2437/3399292809.py:19: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  reviews["review_creation_date"]      = pd.to_datetime(reviews["review_creation_date"],      infer_datetime_format=True)
/tmp/ipykernel_2437/3399292809.py:20: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  reviews["review_answer_timestamp"]  = pd.to_datetime(reviews["review_answer_timestamp"],  infer_datetime_format=True)


,order_id,review_score,review_creation_date,review_answer_timestamp
0,73fc7af87114b39712e6da79b0a377eb,4,2018-01-18,2018-01-18 21:46:59
1,a548910a1c6147796b98fdf73dbeba33,5,2018-03-10,2018-03-11 03:05:13
2,f9e4b658b201a9f2ecdecbb34bed034b,5,2018-02-17,2018-02-18 14:36:24
3,658677c97b385a9be170737859d3511b,5,2017-04-21,2017-04-21 22:02:06
4,8e6bfb81e283fa7e4f11123a3fb894f1,5,2018-03-01,2018-03-02 10:26:53


## Remoção de duplicados  
Vamos garantir que não haja linhas repetidas em nenhum dataset.

In [23]:
# Garanta que todos os DataFrames foram carregados
import pandas as pd

# Se alguma variável não existir, carrega de novo
try:
    customers
except NameError:
    customers = pd.read_csv("../data/olist_customers_dataset.csv")
    print("customers recarregado:", customers.shape)

# Opcional: confira também as outras
for name in ["orders", "products", "items", "reviews"]:
    if name not in globals():
        df = pd.read_csv(f"../data/olist_{name}_dataset.csv") \
               if name != "items" else \
               pd.read_csv("../data/olist_order_items_dataset.csv")
        globals()[name] = df
        print(f"{name} recarregado:", df.shape)


In [24]:
datasets = [orders, products, items, reviews, customers]
names    = ["orders", "products", "items", "reviews", "customers"]

for df, name in zip(datasets, names):
    before = df.shape[0]
    df.drop_duplicates(inplace=True)
    after  = df.shape[0]
    print(f"{name}: {before} → {after} (duplicados removidos: {before-after})")

orders: 96476 → 96476 (duplicados removidos: 0)
products: 32951 → 32951 (duplicados removidos: 0)
items: 112650 → 112650 (duplicados removidos: 0)
reviews: 99224 → 99224 (duplicados removidos: 0)
customers: 99441 → 99441 (duplicados removidos: 0)


## Tratamento de valores nulos  
Decisões iniciais:
- `order_delivered_customer_date`: remover pedidos ainda não entregues.
- `review_score`: preencher com 0 para indicar “sem avaliação”.

In [25]:
# Remover pedidos sem data de entrega ao cliente
orders = orders[~orders["order_delivered_customer_date"].isna()]

# Preencher review_score faltante
reviews["review_score"].fillna(0, inplace=True)

# Verificar nulos remanescentes
print("Nulos em orders:\n", orders.isna().sum(), "\n")
print("Nulos em reviews:\n", reviews.isna().sum(), "\n")


Nulos em orders:
 order_id                          0
customer_id                       0
order_status                      0
order_purchase_timestamp          0
order_approved_at                14
order_delivered_carrier_date      1
order_delivered_customer_date     0
order_estimated_delivery_date     0
dtype: int64 

Nulos em reviews:
 order_id                   0
review_score               0
review_creation_date       0
review_answer_timestamp    0
dtype: int64 



## Criação de métricas derivadas  
- **delivery_time**: dias entre compra e entrega ao cliente.  
- **order_value**: valor total por item multiplicado pela quantidade.  

## Conversão de colunas de data e ajuste de `total_price`

In [26]:
# — Garantir formato datetime em todas as datas de orders
date_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_customer_date",
    "order_delivered_carrier_date",
    "order_estimated_delivery_date"
]
for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors="coerce")

# — Ajustar total_price usando apenas o price (cada item é unitário)
items["total_price"] = items["price"]

# Verificar tipos e amostras
print(orders[date_cols].dtypes)
display(items.head())


order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,total_price
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,58.90
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,239.90
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,199.00
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,12.99
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,199.90


In [27]:
# Tempo de entrega em dias (já com colunas datetime)
orders["delivery_time"] = (
    orders["order_delivered_customer_date"]
    - orders["order_purchase_timestamp"]
).dt.days

display(orders[["order_id","order_purchase_timestamp","order_delivered_customer_date","delivery_time"]].head())

,order_id,order_purchase_timestamp,order_delivered_customer_date,delivery_time
0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,2017-10-10 21:25:13,8
1,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,2018-08-07 15:27:45,13
2,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:38:49,2018-08-17 18:06:29,9
3,949d5b44dbf5de918fe9c16f97b45f8a,2017-11-18 19:28:06,2017-12-02 00:28:42,13
4,ad21c59c0840e6cb83a9ceb5573f8159,2018-02-13 21:18:39,2018-02-16 18:17:02,2


## Merge dos datasets  
Unimos na ordem: orders → items → products → customers → reviews.

In [28]:
df = (
    orders.merge(items,     on="order_id",   how="inner")
          .merge(products,  on="product_id", how="left")
          .merge(customers, on="customer_id",how="left")
          .merge(reviews,   on="order_id",   how="left")
)
print("DataFrame final:", df.shape)
display(df.head())

DataFrame final: (110839, 31)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_time,order_item_id,...,product_length_cm,product_height_cm,product_width_cm,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,review_score,review_creation_date,review_answer_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8,1,...,19.0,8.0,13.0,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,4.0,2017-10-11,2017-10-12 03:43:48
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13,1,...,19.0,13.0,19.0,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,4.0,2018-08-08,2018-08-08 18:37:50
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9,1,...,24.0,19.0,21.0,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,5.0,2018-08-18,2018-08-22 19:07:58
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,13,1,...,30.0,10.0,20.0,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,5.0,2017-12-03,2017-12-05 19:21:58
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2,1,...,51.0,15.0,15.0,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,5.0,2018-02-17,2018-02-18 13:02:51


## Salvando o dataset limpo  
Para facilitar reuso, vamos gravar o DataFrame único em CSV.

In [29]:
df.to_csv("../data/ecommerce_clean.csv", index=False)
print("Arquivo salvo em ../data/ecommerce_clean.csv")

Arquivo salvo em ../data/ecommerce_clean.csv
